In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import json
import csv
import psycopg2

if not os.path.exists(os.path.join(os.getcwd(), 'config.json')):
  # preventing to go always to the parent folder everytime we run the jupyter notebook without restarting
  os.chdir("..")
if not os.getcwd() in sys.path:
  sys.path.append(os.getcwd())

from graph import Graph

config = None
with open('config.json') as config_file:
  config = json.load(config_file)

# ---------------------------------------------------
# config["database"]["database"] = "wikilite" # WIKILITE
# ---------------------------------------------------


In [2]:
graph = Graph(config)
graph.connect()

In [3]:
# Compute the words that are shared between the seeded document 
# (But has to be generalized with N pages so we can do for all words of all seeds pages)
# graph.cursor.execute("drop table if exists idf2")
# graph.conn.commit()
# graph.cursor.execute("""
#   create table idf2 as (
#     select distinct grams as gram, 0 as df, 0 as idf
#     from nodes_info, grams(nodes_info.content)
#     where id < 100000
#   );
# """)
# graph.conn.commit()

In [3]:
graph.close()

In [4]:
all_target_nodes = []
with open('data/contrasting_arguments_labeled_stats_selected.csv', 'r') as f:
  reader = csv.DictReader(f)
  for r in reader:
    if int(r['Selected']) != 1:
      continue
    all_target_nodes.append(int(r['Mitigator_ID']))
    all_target_nodes.append(int(r['Disinformer_ID']))

print("seeded pages:", len(all_target_nodes))
# all_target_nodes = all_target_nodes[:2]

seeded pages: 8


In [5]:
conn = psycopg2.connect(**config['database'])
cursor = conn.cursor()

In [6]:
cursor.execute("select compute_idf_seeds_nodes(%s)", (all_target_nodes,))
print("Important term saved:", cursor.fetchall())
conn.commit()

Important term saved: [('',)]


In [ ]:
# We don't do for all nodes, but only the target nodes we care about in our experiments
# To do for all:

# cursor.execute("select compute_idf()")
# print("Important term saved:", cursor.fetchall())
# conn.commit()

# To complete estimation of 166 hours to complete (about 7 days) 
# assuming is linear complexity but never tried

In [9]:
# Compute the words that are shared between the seeded document 
# (But has to be generalized with N pages so we can do for all words of all seeds pages)
# cursor.execute("drop table if exists tf_seeds_pages")
conn.commit()
cursor.execute("""
  drop table if exists tf_seeds_pages;
  create table tf_seeds_pages as (
    select distinct grams as gram, 0 as df, 0 as idf
    from nodes_info, grams(nodes_info.content)
    where nodes_info.id = any(%s)
  );
""", [all_target_nodes])
conn.commit()

cursor.execute("""
  select count(*) from tf_seeds_pages
""")
print("Common words among seeds pages:", cursor.fetchone()[0])

Common words among seeds pages: 3830


In [10]:
cursor.close()
conn.close()